In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import datetime
import re
import sqlalchemy as sa
from sqlalchemy import create_engine
import pymysql
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
#creating connection with MySQL datbase

user = "brenobertolucci"
password = "Panda190322"
url_banco = "brenobertolucci.mysql.pythonanywhere-services.com"
nome_db = "brenobertolucci$dados_anbima"
conn_str = f"mysql+pymysql://{user}:{password}@{url_banco}/{nome_db}"
engine = create_engine(conn_str)

In [ ]:
def read_sql(query, engine):
    with engine.begin() as conn:
        results = pd.read_sql_query(sa.text(query), conn)
    return results
    
def execute_sql(query, engine):
    with engine.begin() as conn:
        results = conn.execute(
            sa.text(query)
            )
    return results

def df_to_sql(df, table, engine):
    with engine.begin() as conn:
        df.to_sql(table ,conn, index = False, index_label = False, if_exists = "append") 

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())

In [ ]:
#importing full df 
tabela_anbima = read_sql('select * from dados_anbima',engine)

#selecting unique tickers
lista_tickers = tabela_anbima["Ticker"].unique()

In [ ]:
#webscrapping the sectors of all tickers

dict_setores ={}

for ticker in lista_tickers:
    try:
        driver.get("https://data.anbima.com.br/debentures")
        driver.implicitly_wait(20)
        barra_busca = driver.find_element(By.XPATH, "/html/body/div[2]/header/main/div/form/input")
        barra_busca.clear()
        barra_busca.send_keys(ticker)
        barra_busca.send_keys(Keys.ENTER)
        setor = driver.find_element(By.XPATH, "/html/body/div[2]/main/div[2]/div[1]/div[2]/ul/li/div[3]/div[1]/dl/dd").text
        dict_setores[ticker] = setor
    except:
         dict_setores[ticker] = "Erro"


In [ ]:
#creating a DF with tickers/sectors

lista_setores =[]
lista_tickers =[]

for ticker in dict_setores:
    lista_setores.append(dict_setores[ticker])
    lista_tickers.append(ticker)
    
    
tickers_setores = pd.DataFrame.from_dict({"Ticker":lista_tickers , "Setor":lista_setores})
tickers_setores

In [ ]:
#checking for erros
mascara_erro = tickers_setores["Setor"] == "Erro"
mascara_erro2 = tickers_setores["Setor"] == "-"
tickers_setores[mascara_erro|mascara_erro2]

In [ ]:
#correcting erros

tickers_setores.iloc[76,1] = "Transporte e Logística"
tickers_setores.iloc[392,1] = "Transporte e Logística"
tickers_setores.iloc[259,1] = "Papel e Celulose"

In [ ]:
#exporting files:

#sql

df_to_sql(setores,"setores_anbima",engine)

In [ ]:
tickers_setores.to_excel("data/setores.xlsx")

In [ ]:
setores = pd.read_excel("daily_files/setores.xlsx")

In [ ]:
df_teste = read_sql('select * from setores_anbima',engine)

In [ ]:
df_teste